In [20]:
# Copyright (c) OpenMMLab. All rights reserved.
from operator import attrgetter
from typing import List, Union

import torch
import torch.nn as nn


def efficient_conv_bn_eval_forward(bn: nn.modules.batchnorm._BatchNorm,
                                   conv: nn.modules.conv._ConvNd,
                                   x: torch.Tensor):
    """Code borrowed from mmcv 2.0.1, so that this feature can be used for old
    mmcv versions.

    Implementation based on https://arxiv.org/abs/2305.11624
    "Tune-Mode ConvBN Blocks For Efficient Transfer Learning"
    It leverages the associative law between convolution and affine transform,
    i.e., normalize (weight conv feature) = (normalize weight) conv feature.
    It works for Eval mode of ConvBN blocks during validation, and can be used
    for training as well. It reduces memory and computation cost.
    Args:
        bn (_BatchNorm): a BatchNorm module.
        conv (nn._ConvNd): a conv module
        x (torch.Tensor): Input feature map.
    """
    # These lines of code are designed to deal with various cases
    # like bn without affine transform, and conv without bias
    weight_on_the_fly = conv.weight
    if conv.bias is not None:
        bias_on_the_fly = conv.bias
    else:
        bias_on_the_fly = torch.zeros_like(bn.running_var)

    if bn.weight is not None:
        bn_weight = bn.weight
    else:
        bn_weight = torch.ones_like(bn.running_var)

    if bn.bias is not None:
        bn_bias = bn.bias
    else:
        bn_bias = torch.zeros_like(bn.running_var)

    # shape of [C_out, 1, 1, 1] in Conv2d
    weight_coeff = torch.rsqrt(bn.running_var +
                               bn.eps).reshape([-1] + [1] *
                                               (len(conv.weight.shape) - 1))
    # shape of [C_out, 1, 1, 1] in Conv2d
    coefff_on_the_fly = bn_weight.view_as(weight_coeff) * weight_coeff

    # shape of [C_out, C_in, k, k] in Conv2d
    weight_on_the_fly = weight_on_the_fly * coefff_on_the_fly
    # shape of [C_out] in Conv2d
    bias_on_the_fly = bn_bias + coefff_on_the_fly.flatten() *\
        (bias_on_the_fly - bn.running_mean)

    return conv._conv_forward(x, weight_on_the_fly, bias_on_the_fly)


def efficient_conv_bn_eval_control(bn: nn.modules.batchnorm._BatchNorm,
                                   conv: nn.modules.conv._ConvNd,
                                   x: torch.Tensor):
    """This function controls whether to use `efficient_conv_bn_eval_forward`.

    If the following `bn` is in `eval` mode, then we turn on the special
    `efficient_conv_bn_eval_forward`.
    """
    if not bn.training:
        # bn in eval mode
        output = efficient_conv_bn_eval_forward(bn, conv, x)
        return output
    else:
        conv_out = conv._conv_forward(x, conv.weight, conv.bias)
        return bn(conv_out)


def efficient_conv_bn_eval_graph_transform(fx_model):
    """Find consecutive conv+bn calls in the graph, inplace modify the graph
    with the fused operation."""
    modules = dict(fx_model.named_modules())

    patterns = [(torch.nn.modules.conv._ConvNd,
                 torch.nn.modules.batchnorm._BatchNorm)]

    pairs = []
    # Iterate through nodes in the graph to find ConvBN blocks
    for node in fx_model.graph.nodes:
        # If our current node isn't calling a Module then we can ignore it.
        if node.op != 'call_module':
            continue
        target_module = modules[node.target]
        found_pair = False
        for conv_class, bn_class in patterns:
            if isinstance(target_module, bn_class):
                source_module = modules[node.args[0].target]
                if isinstance(source_module, conv_class):
                    found_pair = True
        # Not a conv-BN pattern or output of conv is used by other nodes
        if not found_pair or len(node.args[0].users) > 1:
            continue

        # Find a pair of conv and bn computation nodes to optimize
        conv_node = node.args[0]
        bn_node = node
        pairs.append([conv_node, bn_node])

    for conv_node, bn_node in pairs:
        # set insertion point
        fx_model.graph.inserting_before(conv_node)
        # create `get_attr` node to access modules
        # note that we directly call `create_node` to fill the `name`
        # argument. `fx_model.graph.get_attr` and
        # `fx_model.graph.call_function` does not allow the `name` argument.
        conv_get_node = fx_model.graph.create_node(
            op='get_attr', target=conv_node.target, name='get_conv')
        bn_get_node = fx_model.graph.create_node(
            op='get_attr', target=bn_node.target, name='get_bn')
        # prepare args for the fused function
        args = (bn_get_node, conv_get_node, conv_node.args[0])
        # create a new node
        new_node = fx_model.graph.create_node(
            op='call_function',
            target=efficient_conv_bn_eval_control,
            args=args,
            name='efficient_conv_bn_eval')
        # this node replaces the original conv + bn, and therefore
        # should replace the uses of bn_node
        bn_node.replace_all_uses_with(new_node)
        # take care of the deletion order:
        # delete bn_node first, and then conv_node
        fx_model.graph.erase_node(bn_node)
        fx_model.graph.erase_node(conv_node)

    # regenerate the code
    fx_model.graph.lint()
    fx_model.recompile()


def turn_on_efficient_conv_bn_eval_for_single_model(model: torch.nn.Module):
    import torch.fx as fx

    # currently we use `fx.symbolic_trace` to trace models.
    # in the future, we might turn to pytorch 2.0 compile infrastructure to
    # get the `fx.GraphModule` IR. Nonetheless, the graph transform function
    # can remain unchanged. We just need to change the way
    # we get `fx.GraphModule`.
    fx_model: fx.GraphModule = fx.symbolic_trace(model)
    efficient_conv_bn_eval_graph_transform(fx_model)
    model.forward = fx_model.forward


def turn_on_efficient_conv_bn_eval(model: torch.nn.Module,
                                   modules: Union[List[str], str]):
    if isinstance(modules, str):
        modules = [modules]
    for module_name in modules:
        module = attrgetter(module_name)(model)
        turn_on_efficient_conv_bn_eval_for_single_model(module)

In [44]:
import torch
import torch.nn as nn
import torch.fx as fx

# 定义一个简单的卷积神经网络作为示例
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc = nn.Linear(32 * 32 * 32, 10)  # 假设输入图像大小为32x32

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)  # 展平
        x = self.fc(x)
        return x

class SimpleCNN2(nn.Module):
    def __init__(self):
        super(SimpleCNN2 ,self).__init__()
        self.backbone = SimpleCNN()

    def forward(self, x):
        x = self.backbone(x)
        return x
# 调用模型并检查优化情况
model = SimpleCNN2()

# 创建一个随机输入
input_tensor = torch.randn(1, 3, 32, 32)

# 调用函数以启用高效的Conv + BN融合并打印优化前后的计算图


model = model.eval()
turn_on_efficient_conv_bn_eval(model, ['backbone'])
print(model.forward)
# 执行前向传播
output = model(input_tensor)

# 打印输出的形状
print(f"Output shape: {output.shape}")


<bound method SimpleCNN2.forward of SimpleCNN2(
  (backbone): SimpleCNN(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc): Linear(in_features=32768, out_features=10, bias=True)
  )
)>
Output shape: torch.Size([1, 10])


In [29]:
# 检查替换后的 forward 方法
print(model.forward)


<bound method forward of SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32768, out_features=10, bias=True)
)>


In [30]:
# 打印 FX 计算图
import torch.fx as fx
fx_model = fx.symbolic_trace(model)
print(fx_model.graph)


graph():
    %x : [num_users=1] = placeholder[target=x]
    %conv1 : [num_users=1] = call_module[target=conv1](args = (%x,), kwargs = {})
    %bn1 : [num_users=1] = call_module[target=bn1](args = (%conv1,), kwargs = {})
    %relu : [num_users=1] = call_function[target=torch.relu](args = (%bn1,), kwargs = {})
    %conv2 : [num_users=1] = call_module[target=conv2](args = (%relu,), kwargs = {})
    %bn2 : [num_users=1] = call_module[target=bn2](args = (%conv2,), kwargs = {})
    %relu_1 : [num_users=2] = call_function[target=torch.relu](args = (%bn2,), kwargs = {})
    %size : [num_users=1] = call_method[target=size](args = (%relu_1, 0), kwargs = {})
    %view : [num_users=1] = call_method[target=view](args = (%relu_1, %size, -1), kwargs = {})
    %fc : [num_users=1] = call_module[target=fc](args = (%view,), kwargs = {})
    return fc


In [31]:
# 打印 FX 计算图
import torch.fx as fx
fx_model = fx.symbolic_trace(model)
print(fx_model.graph)


graph():
    %x : [num_users=1] = placeholder[target=x]
    %conv1 : [num_users=1] = call_module[target=conv1](args = (%x,), kwargs = {})
    %bn1 : [num_users=1] = call_module[target=bn1](args = (%conv1,), kwargs = {})
    %relu : [num_users=1] = call_function[target=torch.relu](args = (%bn1,), kwargs = {})
    %conv2 : [num_users=1] = call_module[target=conv2](args = (%relu,), kwargs = {})
    %bn2 : [num_users=1] = call_module[target=bn2](args = (%conv2,), kwargs = {})
    %relu_1 : [num_users=2] = call_function[target=torch.relu](args = (%bn2,), kwargs = {})
    %size : [num_users=1] = call_method[target=size](args = (%relu_1, 0), kwargs = {})
    %view : [num_users=1] = call_method[target=view](args = (%relu_1, %size, -1), kwargs = {})
    %fc : [num_users=1] = call_module[target=fc](args = (%view,), kwargs = {})
    return fc


In [28]:
# 检查替换后的 forward 方法
print(model.forward)


<bound method forward of SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32768, out_features=10, bias=True)
)>


In [20]:
print(output_before_fusion)

tensor([[[[-0.4928, -0.7771, -1.0377,  ..., -0.8974, -0.3298, -0.3931],
          [ 0.2901,  0.2581,  0.7802,  ...,  0.9171,  0.9820,  0.1806],
          [-1.3592,  0.2125,  1.1953,  ..., -0.4506,  0.3617,  1.6183],
          ...,
          [-0.5671,  0.9266,  2.0484,  ..., -0.4558, -0.8388,  0.0452],
          [ 0.0935, -0.7979,  0.9503,  ...,  0.5598, -1.9626, -0.2911],
          [ 0.1995,  0.6507, -0.2142,  ..., -0.3146, -0.2791,  0.4547]],

         [[ 1.4356,  0.9129, -0.5031,  ..., -0.1020,  1.0058, -0.4193],
          [-1.4852, -0.4363, -0.8256,  ..., -0.9477, -0.0374,  0.1647],
          [-0.3652, -0.2904,  2.5691,  ...,  0.5813, -0.8416, -0.9438],
          ...,
          [ 0.4869,  2.5273,  1.1332,  ...,  2.2675,  0.0770,  0.6853],
          [ 0.0430, -1.8617, -0.6892,  ..., -0.8342, -0.1863, -0.8786],
          [ 0.0802,  0.3856, -0.6600,  ...,  0.8860,  1.6499, -0.8380]],

         [[-0.1324,  0.0349,  0.6080,  ..., -0.8111,  0.4878, -0.4131],
          [-0.3960, -0.8790, -

In [21]:
print(output_after_fusion)

tensor([[[[-0.2322, -0.3684, -0.4932,  ..., -0.4260, -0.1541, -0.1844],
          [ 0.1428,  0.1274,  0.3775,  ...,  0.4430,  0.4741,  0.0903],
          [-0.6471,  0.1056,  0.5763,  ..., -0.2120,  0.1770,  0.7788],
          ...,
          [-0.2678,  0.4476,  0.9848,  ..., -0.2145, -0.3979,  0.0255],
          [ 0.0486, -0.3783,  0.4589,  ...,  0.2719, -0.9361, -0.1356],
          [ 0.0993,  0.3154, -0.0988,  ..., -0.1468, -0.1298,  0.2216]],

         [[ 0.8490,  0.5423, -0.2886,  ..., -0.0532,  0.5968, -0.2394],
          [-0.8648, -0.2494, -0.4778,  ..., -0.5494, -0.0153,  0.1032],
          [-0.2077, -0.1638,  1.5140,  ...,  0.3477, -0.4872, -0.5472],
          ...,
          [ 0.2923,  1.4895,  0.6715,  ...,  1.3371,  0.0518,  0.4087],
          [ 0.0319, -1.0857, -0.3978,  ..., -0.4829, -0.1027, -0.5089],
          [ 0.0537,  0.2329, -0.3806,  ...,  0.5265,  0.9747, -0.4851]],

         [[-0.0990,  0.0130,  0.3966,  ..., -0.5533,  0.3161, -0.2869],
          [-0.2754, -0.5987, -

In [26]:
# Copyright (c) OpenMMLab. All rights reserved.
import unittest
from unittest import TestCase

import torch
from torch import nn





class BackboneModel(nn.Module):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.conv1 = nn.Conv2d(6, 6, 6)
        self.bn1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 6, 6)
        self.bn2 = nn.BatchNorm2d(6)
        self.conv3 = nn.Conv2d(6, 6, 6)
        self.bn3 = nn.BatchNorm2d(6)

    def forward(self, x):

        # this conv-bn pair can use efficient_conv_bn_eval feature
        x = self.bn1(self.conv1(x))
        # this conv-bn pair can use efficient_conv_bn_eval feature
        # only for the second `self.conv2` call.
        x = self.bn2(self.conv2(self.conv2(x)))
        # this conv-bn pair can use efficient_conv_bn_eval feature
        # just for the first forward of the `self.bn3`
        x = self.bn3(self.bn3(self.conv3(x)))
        return x




model = BackboneModel()
model.eval()
input = torch.randn(64, 6, 32, 32)
output = model(input)
turn_on_efficient_conv_bn_eval_for_single_model(model)
output2 = model(input)
print((output - output2).abs().max().item())


2.0116567611694336e-07


In [ ]:
import torch
import torch.nn as nn

class ConvBn(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups=1):
        super(ConvBn, self).__init__()
        self.in_channels = in_channels
        self.groups = groups
        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
    @torch.no_grad()
    def switch_to_deploy(self):
        kernel = self.conv.weight
        gamma = self.bn.weight
        beta = self.bn.bias
        running_mean = self.bn.running_mean
        running_var = self.bn.running_var
        eps = self.bn.eps

        std = (running_var + eps).sqrt()
        t = (gamma / std)[:, None, None, None]

        # 融合卷积权重
        fused_conv_weight = kernel * t

        # 融合卷积偏置
        fused_conv_bias = beta - running_mean * gamma / std
        fused_conv_bias = nn.Parameter(fused_conv_bias)

        self.rbr_reparam = nn.Conv2d(self.conv.in_channels, self.conv.out_channels,
                                      kernel_size=self.conv.kernel_size, stride=self.conv.stride,
                                      padding=self.conv.padding, dilation=self.conv.dilation,
                                      groups=self.conv.groups, bias=True)

        self.rbr_reparam.weight.data = fused_conv_weight
        self.rbr_reparam.bias = fused_conv_bias

        del self.bn
        del self.conv

    def forward(self, x):
        if hasattr(self, 'rbr_reparam'):
            return self.rbr_reparam(x)
        x = self.conv(x)
        x = self.bn(x)
        return x


conv_bn_layer = ConvBn(3, 16, 3, 1, 1)
input_tensor = torch.randn(1, 3, 32, 32)

output_before_fusion = conv_bn_layer(input_tensor)
conv_bn_layer.switch_to_deploy()
output_after_fusion = conv_bn_layer(input_tensor)

print("Output before fusion: ", output_before_fusion.shape)
print("Output after fusion: ", output_after_fusion.shape)
print("Are the outputs close? ", torch.allclose(output_before_fusion, output_after_fusion, atol=1e-6))


Output before fusion:  torch.Size([1, 16, 32, 32])
Output after fusion:  torch.Size([1, 16, 32, 32])
Are the outputs close?  False
